In [1]:
!pip install happytransformer

In [2]:
from happytransformer import  HappyTextToText
from difflib import SequenceMatcher
from happytransformer import TTSettings
import pandas as pd
import numpy as np

In [3]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

07/07/2022 09:54:04 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [4]:
  beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=100)

In [5]:
def Grammer_score(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [7]:
df = pd.read_csv('review_data.csv')
df = df.sample(frac=0.003) #sampling a fraction of data to reduce time
df.head()

,text,star,app_id,reviewDate
631,Nice,5,com.gameloft.android.ANMP.GloftA8HM,18/03/21
2714,ABBAS Zohrain,1,com.king.candycrushsodasaga,18/03/21
9608,"Lately, this app often interrupt and discontin...",2,com.tencent.ibg.joox,19/03/21
29393,Love 💕,5,com.imangi.templerun2,21/03/21
4920,Favorite,5,com.playrix.gardenscapes,18/03/21


In [8]:
correct = []
for text in df.text:
  output = happy_tt.generate_text("grammar: " + text,args=beam_settings)
  correct.append(output.text)
df['Correct_text'] = correct

In [9]:
score = []
for text,Correct_text in zip(df.text,df.Correct_text):
  score.append(Grammer_score(text,Correct_text))
df['score'] = score

In [10]:
df['Grammer'] = np.where(df['score'] > 0.9 , "Correct", "wrong")

In [11]:
df.head()

,text,star,app_id,reviewDate,Correct_text,score,Grammer
631,Nice,5,com.gameloft.android.ANMP.GloftA8HM,18/03/21,Nice. Nice. Nice. Nice. Nice. Nice. Nice. Nice...,0.029963,wrong
2714,ABBAS Zohrain,1,com.king.candycrushsodasaga,18/03/21,ABBAS Zohrain.,0.962963,Correct
9608,"Lately, this app often interrupt and discontin...",2,com.tencent.ibg.joox,19/03/21,"Lately, this app often interrupts and disconti...",0.984615,Correct
29393,Love 💕,5,com.imangi.templerun2,21/03/21,Love ...,0.169492,wrong
4920,Favorite,5,com.playrix.gardenscapes,18/03/21,Favorite thing? Favorite thing? Favorite thing...,0.049080,wrong
